In [19]:
# Install the required libraries
%pip install youtube-transcript-api
%pip install openai==0.28

In [40]:
import openai
from getpass import getpass
from youtube_transcript_api import YouTubeTranscriptApi

# Securely input your OpenAI API key
api_key = getpass("Enter your OpenAI API key: ")
openai.api_key = api_key

def get_video_id(url):
    from urllib.parse import urlparse, parse_qs
    query = urlparse(url).query
    video_id = parse_qs(query).get('v')
    return video_id[0] if video_id else None

def fetch_transcript(video_id):
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = ' '.join([item['text'] for item in transcript_list])
        return transcript_text
    except Exception as e:
        print("Failed to fetch transcript:", str(e))
        return None

def summarize_text(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant tasked with providing a comprehensive summary that covers all key points in detail."},
                {"role": "user", "content": text}
            ],
            temperature=0.5,  # Slightly more creative
            max_tokens=600    # Allow for a longer response
        )
        summary = response.choices[0].message['content']  # Correct attribute access for chat responses
        return summary
    except Exception as e:
        print("Error during summarization:", str(e))
        return str(e)

# Example usage
if __name__ == "__main__":
    video_url = input("Enter the YouTube video URL: ")
    video_id = get_video_id(video_url)
    if video_id:
        print("Fetching transcript...")
        transcript = fetch_transcript(video_id)
        if transcript:
            print("Transcript fetched. Generating summary...")
            summary = summarize_text(transcript)
            print("Summary:", summary)
        else:
            print("Failed to fetch transcript.")
    else:
        print("Invalid YouTube URL")

Enter your OpenAI API key: ··········
Enter the YouTube video URL: https://www.youtube.com/watch?v=I4pQbo5MQOs
Fetching transcript...
Transcript fetched. Generating summary...
Summary: The sentence "This statement is false" creates a paradox as it refers to itself directly, leading to the question of its truth value. This thought experiment led Kurt Gödel to a groundbreaking discovery in the early 20th century regarding the limitations of mathematical proofs. Mathematical proofs are logical arguments based on axioms, which are fundamental statements about numbers. Gödel's work focused on the idea that some true mathematical statements are unprovable within a given set of axioms.

Gödel developed a method to code mathematical statements into numbers, allowing mathematics to talk about itself. He constructed the first self-referential mathematical statement, "This statement cannot be proved," which led to his Incompleteness Theorem. This theorem asserts that in any axiomatic system, ther

In [37]:
"""
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Define stop words
stop_words = list(set(stopwords.words('english')))

# Create a TF-IDF vectorizer instance with stop words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

# Fit the vectorizer to the summary and transform the text into a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform([summary])

# Get feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a dictionary to store word-frequency pairs
word_frequencies = {}

# Loop through each word and its TF-IDF score
for col in tfidf_matrix.nonzero()[1]:
    word = feature_names[col]
    word_frequencies[word] = tfidf_matrix[0, col]

# Sort the dictionary by TF-IDF score in descending order
sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)

# Print the top N words and their TF-IDF scores
top_n = 5
print("Top", top_n, "keywords:")
for word, score in sorted_word_frequencies[:top_n]:
    print(word, "-", score)"""

Top 5 keywords:
mathematical - 0.40993876680684926
gödel - 0.3643900149394215
statements - 0.31884126307199384
mathematics - 0.22774375933713847
true - 0.22774375933713847


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
!pip install google

In [38]:
"""
import nltk
import requests
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from bs4 import BeautifulSoup
from googlesearch import search

def find_resources(word):
    try:
        search_results = search(word, num=1, stop=1, pause=2.0)
        first_result_url = next(search_results)
        print(f"URL of the first search result for '{word}':")
        print(first_result_url)
    except StopIteration:
        print("No search results found.")

# Sample text
text = summary
# Tokenize the text into words
words = word_tokenize(text)

# Filter out stopwords
stop_words = set(stopwords.words('english'))
filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]

# Calculate frequency distribution
fdist = FreqDist(filtered_words)


# Print the most common words (keywords)
print("Keywords:")
for word, frequency in fdist.most_common(5):
    print(word + ":", frequency)
    find_resources(word)"""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Keywords:
mathematical: 9
URL of the first search result for 'mathematical':
https://www.merriam-webster.com/dictionary/mathematical
gödel: 8
URL of the first search result for 'gödel':
https://en.wikipedia.org/wiki/Kurt_G%C3%B6del
statements: 7
URL of the first search result for 'statements':
https://www.statementstile.com/
true: 5
URL of the first search result for 'true':
https://www.merriam-webster.com/dictionary/true
mathematics: 5
URL of the first search result for 'mathematics':
https://en.wikipedia.org/wiki/Mathematics


In [42]:
import nltk
import requests
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup
from googlesearch import search

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

def get_search_results(word):
    try:
        search_results = search(word, num=5, stop=5, pause=2.0)
        return list(search_results)
    except StopIteration:
        return []

def get_page_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except:
        return ''

def extract_keywords(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    fdist = FreqDist(filtered_words)
    return [word for word, _ in fdist.most_common(5)]

def classify_relevance(search_results, keywords):
    relevance = []
    for result in search_results:
        title = result.split(' - ')[0].lower()
        is_relevant = any(keyword in title for keyword in keywords)
        relevance.append(1 if is_relevant else 0)
    return relevance

def train_classifier(X, y):
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(X)
    clf = LogisticRegression()
    clf.fit(X_train, y)
    return clf, vectorizer

def find_resources(text):
    keywords = extract_keywords(text)
    print(keywords)
    search_results = get_search_results(' '.join(keywords))
    relevance_labels = classify_relevance(search_results, keywords)
    X_train, _, y_train, _ = train_test_split(search_results, relevance_labels, test_size=0.2, random_state=42)
    clf, vectorizer = train_classifier(X_train, y_train)
    X_test = vectorizer.transform(search_results)
    predicted_labels = clf.predict(X_test)
    relevant_results = [result for result, label in zip(search_results, predicted_labels) if label == 1]
    return relevant_results

# Find relevant resources
relevant_resources = find_resources(summary)
print("Relevant resources:")
for resource in relevant_resources:
    print(resource)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['mathematical', 'gödel', 'statements', 'unprovable', 'statement']
Relevant resources:
https://www.reddit.com/r/math/comments/ugzwg1/unprovable_true_statements/
https://www.reddit.com/r/compsci/comments/upi7g7/is_there_a_way_to_prove_that_a_specific_statement/
https://www.reddit.com/r/math/comments/bx4lkh/do_we_have_any_idea_how_many_true_but_unprovable/
